**Data Sources (as per Part 3 requirements):**
- BLS Employment Data: `s3://rearc-deepa-demo/raw/pr/pr.data.0.Current`
- DataUSA Population: `s3://rearc-deepa-demo/raw/datausa/population/`

**Analyses Performed:**
1. Population statistics for years 2013-2018 (mean and standard deviation)
2. Best year per BLS series (year with maximum sum of quarterly values)
3. Series PRS30006032 Period Q01 joined with population data

In [44]:
!pip install boto3 awscrt

In [45]:
import pandas as pd
import boto3
import json
from io import StringIO, BytesIO
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"Boto3 version: {boto3.__version__}")

✓ Libraries imported successfully
Pandas version: 2.2.3
Boto3 version: 1.42.14


## 2. Configure S3 Client

In [46]:
import subprocess
import os

# Run AWS SSO login
aws_path = r"C:\Program Files\Amazon\AWSCLIV2\aws.exe"
if os.path.exists(aws_path):
    result = subprocess.run([aws_path, 'sso', 'login'], capture_output=True, text=True)
    print(result.stdout)
    if result.stderr:
        print(result.stderr)
    print("\n✓ AWS login completed. Browser should have opened for authentication.")
else:
    print("⚠️ AWS CLI not found. Please run 'aws sso login' in a terminal window.")



Missing the following required SSO configuration values: sso_start_url, sso_region. To make sure this profile is properly configured to use SSO, please run: aws configure sso


✓ AWS login completed. Browser should have opened for authentication.


In [47]:

s3_client = boto3.client('s3', region_name='eu-north-1')
bucket_name = 'rearc-deepa-demo'

print(f"✓ S3 client configured for bucket: {bucket_name}")


✓ S3 client configured for bucket: rearc-deepa-demo


## 3. Load BLS Employment Data

Load the BLS pr.data.0.Current file (tab-delimited CSV with employment metrics)

In [48]:
# Load BLS data file
bls_data_key = 'raw/pr/pr.data.0.Current'
obj = s3_client.get_object(Bucket=bucket_name, Key=bls_data_key)

In [49]:
obj

{'ResponseMetadata': {'RequestId': 'YJDJ14QYH19XNTXK',
  'HostId': 'Ly7t2y/T4tE//osmfKSQmBId4HJUbbCByOFcDMKax76KpmqtJlHvFDwzodVyVuFjyDSYNU41aWbQySgfbr/ctIi6G/NQInAz',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ly7t2y/T4tE//osmfKSQmBId4HJUbbCByOFcDMKax76KpmqtJlHvFDwzodVyVuFjyDSYNU41aWbQySgfbr/ctIi6G/NQInAz',
   'x-amz-request-id': 'YJDJ14QYH19XNTXK',
   'date': 'Mon, 22 Dec 2025 13:40:26 GMT',
   'last-modified': 'Sun, 21 Dec 2025 07:15:31 GMT',
   'etag': '"1d7786e2769dae2411cffb668e1c0466"',
   'x-amz-checksum-crc32': 'RAtGZQ==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'x-amz-server-side-encryption': 'AES256',
   'x-amz-meta-source-last-modified': '2025-09-04T07:30:00+00:00',
   'x-amz-meta-source-url': 'https://download.bls.gov/pub/time.series/pr/pr.data.0.Current',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'content-length': '1564284',
   'server': 'AmazonS3'},
  'ChecksumAlgorithm': 'crc32',
  'RetryAttempts': 0},
 'AcceptRanges': '

In [50]:

bls_data = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8')), sep='\t')

# Clean column names (remove extra whitespace)
bls_data.columns = bls_data.columns.str.strip()

print(f"✓ Loaded BLS data: {len(bls_data):,} rows")
print(f"✓ Columns: {list(bls_data.columns)}")
print("\nFirst few rows:")
bls_data.head()

✓ Loaded BLS data: 37,239 rows
✓ Columns: ['series_id', 'year', 'period', 'value', 'footnote_codes']

First few rows:


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [51]:
# Check null count in all columns
print("NULL VALUES COUNT:")
print("=" * 30)
null_counts = bls_data.isnull().sum()
for col in bls_data.columns:
    null_count = null_counts[col]
    total_rows = len(bls_data)
    percentage = (null_count / total_rows) * 100
    print(f"{col}: {null_count:,} ({percentage:.1f}%)")

print(f"\nTotal rows: {total_rows:,}")
print(f"Total null values across all columns: {null_counts.sum():,}")

NULL VALUES COUNT:
series_id: 0 (0.0%)
year: 0 (0.0%)
period: 0 (0.0%)
value: 0 (0.0%)
footnote_codes: 37,052 (99.5%)

Total rows: 37,239
Total null values across all columns: 37,052


In [52]:
print(f"\n1. BASIC INFO:")
print(f"   Shape: {bls_data.shape}")
print(f"   Columns: {list(bls_data.columns)}")


1. BASIC INFO:
   Shape: (37239, 5)
   Columns: ['series_id', 'year', 'period', 'value', 'footnote_codes']


In [53]:
print(f"DISTINCT VALUES IN ALL COLUMNS:")
print("="*50)

for col in bls_data.columns:
    unique_count = bls_data[col].nunique()
    print(f"\n{col.upper()}:")
    print(f"  Unique values: {unique_count:,}")
    
    if unique_count <= 20:
        # Show all values if 20 or fewer
        unique_vals = bls_data[col].value_counts().head(20)
        for val, count in unique_vals.items():
            print(f"    {val}: {count:,}")
    else:
        # Show top 10 most frequent values
        print(f"  Top 10 most frequent:")
        top_vals = bls_data[col].value_counts().head(10)
        for val, count in top_vals.items():
            print(f"    {val}: {count:,}")

DISTINCT VALUES IN ALL COLUMNS:

SERIES_ID:
  Unique values: 282
  Top 10 most frequent:
    PRS88003203      : 152
    PRS30006011      : 152
    PRS30006012      : 152
    PRS30006013      : 152
    PRS30006021      : 152
    PRS30006022      : 152
    PRS30006023      : 152
    PRS30006031      : 152
    PRS30006032      : 152
    PRS30006033      : 152

YEAR:
  Unique values: 31
  Top 10 most frequent:
    1995: 1,230
    1996: 1,230
    1997: 1,230
    1998: 1,230
    1999: 1,230
    2000: 1,230
    2001: 1,230
    2002: 1,230
    2003: 1,230
    2004: 1,230

PERIOD:
  Unique values: 5
    Q05: 8,325
    Q01: 7,347
    Q02: 7,347
    Q03: 7,110
    Q04: 7,110

VALUE:
  Unique values: 11,486
  Top 10 most frequent:
    1.9: 398
    1.3: 396
    1.8: 392
    2.0: 383
    1.7: 364
    2.1: 352
    1.6: 349
    0.3: 346
    1.4: 343
    0.6: 343

FOOTNOTE_CODES:
  Unique values: 1
    R: 187


In [54]:
# Remove Q05 records as they appear to be incorrect
print(f"Before filtering: {len(bls_data):,} records")
print(f"Q05 records to remove: {len(bls_data[bls_data['period'] == 'Q05']):,}")

bls_data = bls_data[bls_data['period'] != 'Q05'].copy()

print(f"After filtering: {len(bls_data):,} records")
print(f"✓ Removed Q05 records (likely incorrect quarterly data)")

# Verify remaining periods
print(f"\nRemaining periods:")
print(bls_data['period'].value_counts().sort_index())

Before filtering: 37,239 records
Q05 records to remove: 8,325
After filtering: 28,914 records
✓ Removed Q05 records (likely incorrect quarterly data)

Remaining periods:
period
Q01    7347
Q02    7347
Q03    7110
Q04    7110
Name: count, dtype: int64


In [80]:
# Check for case sensitivity issues in string columns
print("CASE SENSITIVITY CHECK:")
print("="*50)

for col in bls_data.select_dtypes(include='object').columns:
    raw_count = bls_data[col].nunique()
    upper_count = bls_data[col].str.upper().nunique()
    
    if raw_count != upper_count:
        print(f"\n⚠️ {col}: Case variations detected!")
        print(f"   Raw distinct: {raw_count}")
        print(f"   Uppercase distinct: {upper_count}")
        print(f"   Difference: {raw_count - upper_count} duplicates when uppercased")
        
        # Show examples
        sample_vals = bls_data[col].value_counts().head(10)
        print(f"   Sample values:")
        for val, count in sample_vals.items():
            print(f"      '{val}': {count:,}")
    else:
        print(f"✓ {col}: No case variations (distinct: {raw_count})")

print("="*50)

CASE SENSITIVITY CHECK:
✓ series_id: No case variations (distinct: 237)
✓ period: No case variations (distinct: 4)


In [55]:
# 2. Missing Values
print(f"\n2. MISSING VALUES:")
missing = bls_data.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("   ✓ No missing values")



2. MISSING VALUES:
footnote_codes    28727
dtype: int64


In [56]:
# Check and drop footnote_codes if not useful
if 'footnote_codes' in bls_data.columns:
    null_count = bls_data['footnote_codes'].isnull().sum()
    empty_count = (bls_data['footnote_codes'] == '').sum()
    unique_count = bls_data['footnote_codes'].nunique()
    
    print(f"footnote_codes analysis:")
    print(f"  Null values: {null_count}/{len(bls_data)} ({null_count/len(bls_data)*100:.1f}%)")
    print(f"  Empty strings: {empty_count}")
    print(f"  Unique values: {unique_count}")
    
    # Drop if mostly null/empty or low information value
    if null_count + empty_count > len(bls_data) * 0.8:
        bls_data = bls_data.drop(columns=['footnote_codes'])
        print(f"  ✓ Dropped footnote_codes (mostly null/empty)")
    else:
        print(f"  Sample values: {bls_data['footnote_codes'].value_counts().head()}")
else:
    print("footnote_codes column not found")

footnote_codes analysis:
  Null values: 28727/28914 (99.4%)
  Empty strings: 0
  Unique values: 1
  ✓ Dropped footnote_codes (mostly null/empty)


In [57]:
# 3. Duplicates
print(f"\n3. DUPLICATES:")
dup_count = bls_data.duplicated().sum()
print(f"   Total duplicate rows: {dup_count}")
if dup_count > 0:
    print(f"   ⚠️ Found {dup_count} duplicate rows - consider removing")



3. DUPLICATES:
   Total duplicate rows: 0


In [58]:
# 4. Data Types
print(f"\n4. DATA TYPES:")
print(bls_data.dtypes)


4. DATA TYPES:
series_id     object
year           int64
period        object
value        float64
dtype: object


In [59]:
# 5. Whitespace Issues
print(f"\n5. WHITESPACE ISSUES:")
for col in bls_data.select_dtypes(include='object').columns:
    ws_count = (bls_data[col].str.strip() != bls_data[col]).sum()
    if ws_count > 0:
        print(f"   ⚠️ {col}: {ws_count} rows with leading/trailing whitespace")
 


5. WHITESPACE ISSUES:
   ⚠️ series_id: 28914 rows with leading/trailing whitespace


In [60]:
bls_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28914 entries, 0 to 37238
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   series_id  28914 non-null  object 
 1   year       28914 non-null  int64  
 2   period     28914 non-null  object 
 3   value      28914 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ MB


In [61]:
bls_data.describe(include='all')

,series_id,year,period,value
count,28914,28914.000000,28914,28914.000000
unique,237,NaN,4,NaN
top,PRS30006011,NaN,Q01,NaN
freq,122,NaN,7347,NaN
mean,NaN,2009.754098,NaN,32.345137
std,NaN,8.807109,NaN,44.798592
min,NaN,1995.000000,NaN,-54.300000
25%,NaN,2002.000000,NaN,0.700000
50%,NaN,2010.000000,NaN,3.900000
75%,NaN,2017.000000,NaN,83.596000


In [62]:
bls_data.describe()

,year,value
count,28914.000000,28914.000000
mean,2009.754098,32.345137
std,8.807109,44.798592
min,1995.000000,-54.300000
25%,2002.000000,0.700000
50%,2010.000000,3.900000
75%,2017.000000,83.596000
max,2025.000000,384.600000


In [63]:
# 6. Value Range Analysis
print(f"\n6. VALUE COLUMN ANALYSIS:")
if 'value' in bls_data.columns:
    print(f"   Data type: {bls_data['value'].dtype}")
    print(f"   Min: {bls_data['value'].min()}")
    print(f"   Max: {bls_data['value'].max()}")
    print(f"   Mean: {bls_data['value'].mean():.2f}")
    # Check for non-numeric values if stored as object
    if bls_data['value'].dtype == 'object':
        non_numeric = pd.to_numeric(bls_data['value'], errors='coerce').isnull().sum()
        if non_numeric > 0:
            print(f"   ⚠️ {non_numeric} non-numeric values found")


6. VALUE COLUMN ANALYSIS:
   Data type: float64
   Min: -54.3
   Max: 384.6
   Mean: 32.35
   Mean: 32.35


In [64]:
# 7. Unique Value Counts
print(f"\n7. UNIQUE VALUES:")
for col in ['series_id', 'year', 'period']:
    if col in bls_data.columns:
        print(f"   {col}: {bls_data[col].nunique()} unique values")


7. UNIQUE VALUES:
   series_id: 237 unique values
   year: 31 unique values
   period: 4 unique values
   series_id: 237 unique values
   year: 31 unique values
   period: 4 unique values


In [65]:

recommendations = []
if dup_count > 0:
    recommendations.append("- Remove duplicate rows")
if bls_data.isnull().sum().sum() > 0:
    recommendations.append("- Handle missing values")
for col in bls_data.select_dtypes(include='object').columns:
    if (bls_data[col].str.strip() != bls_data[col]).sum() > 0:
        recommendations.append(f"- Trim whitespace in '{col}' column")
        break
if 'value' in bls_data.columns and bls_data['value'].dtype == 'object':
    recommendations.append("- Convert 'value' column to numeric")

if recommendations:
    for rec in recommendations:
        print(rec)
else:
    print("✓ Data appears clean - no major cleanup needed")
    
print("="*70)

- Trim whitespace in 'series_id' column


In [66]:
# Trim whitespace in series_id and other string columns
for col in bls_data.select_dtypes(include='object').columns:
    bls_data[col] = bls_data[col].str.strip()

print("✓ Trimmed whitespace from all string columns")
print(f"✓ BLS data shape after cleanup: {bls_data.shape}")

✓ Trimmed whitespace from all string columns
✓ BLS data shape after cleanup: (28914, 4)


In [67]:
print(f"\n7. UNIQUE VALUES:")
for col in ['series_id', 'year', 'period']:
    if col in bls_data.columns:
        print(f"   {col}: {bls_data[col].nunique()} unique values")


7. UNIQUE VALUES:
   series_id: 237 unique values
   year: 31 unique values
   period: 4 unique values


In [77]:
# Check year overlap between BLS and population data
print("JOIN INTEGRITY CHECK:")
print("="*70)

bls_years = set(bls_data['year'].unique())

if len(pop_data) > 0:
    pop_years = set(pop_data['Year'].unique())
    common_years = bls_years & pop_years
    bls_only = bls_years - pop_years
    pop_only = pop_years - bls_years
    
    print(f"BLS years: {min(bls_years)}-{max(bls_years)} ({len(bls_years)} years)")
    print(f"Population years: {min(pop_years)}-{max(pop_years)} ({len(pop_years)} years)")
    print(f"Overlapping: {len(common_years)} years")
    print(f"BLS only: {len(bls_only)} years")
    print(f"Population only: {len(pop_only)} years {sorted(pop_only) if pop_only else '✓'}")
    
    # Test the join
    test_join = bls_data.merge(pop_data[['Year', 'Population']], 
                                left_on='year', right_on='Year', how='left')
    matched = test_join['Population'].notna().sum()
    match_pct = matched/len(bls_data)*100
    
    print(f"\nJoin match rate: {matched:,}/{len(bls_data):,} ({match_pct:.1f}%)")
    
    if match_pct < 100:
        missing_years = sorted(test_join[test_join['Population'].isna()]['year'].unique())
        print(f"Missing population for years: {missing_years[:15]}")
else:
    print("No population data loaded")

print("="*70)

JOIN INTEGRITY CHECK:
BLS years: 1995-2025 (31 years)
Population years: 2013-2023 (10 years)
Overlapping: 10 years
BLS only: 21 years
Population only: 0 years ✓

Join match rate: 9,480/28,914 (32.8%)
Missing population for years: [np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009)]


In [78]:
# 8. POPULATION DATA QUALITY CHECKS
print("POPULATION DATA QUALITY CHECKS:")
print("="*70)

if len(pop_data) > 0:
    # Year range
    print(f"1. Year range: {pop_data['Year'].min()} to {pop_data['Year'].max()}")
    
    # Negative/zero populations
    neg_pop = (pop_data['Population'] < 0).sum()
    zero_pop = (pop_data['Population'] == 0).sum()
    print(f"2. Negative populations: {neg_pop}")
    print(f"3. Zero populations: {zero_pop}")
    
    # Reasonable population range (Honolulu metro typically 900k-1M)
    min_pop = pop_data['Population'].min()
    max_pop = pop_data['Population'].max()
    print(f"4. Population range: {min_pop:,} to {max_pop:,}")
    
    # Check for unrealistic jumps (>20% year-over-year change)
    if 'Year' in pop_data.columns and len(pop_data) > 1:
        pop_sorted = pop_data.sort_values('Year')
        pop_sorted['pct_change'] = pop_sorted['Population'].pct_change() * 100
        large_changes = pop_sorted[abs(pop_sorted['pct_change']) > 20]
        if len(large_changes) > 0:
            print(f"5. ⚠️ Large year-over-year changes (>20%): {len(large_changes)} instances")
            print(large_changes[['Year', 'Population', 'pct_change']])
        else:
            print(f"5. ✓ No unusual year-over-year changes")
    
    # Check for duplicate years
    dup_years = pop_data[pop_data.duplicated(subset=['Year'], keep=False)]
    if len(dup_years) > 0:
        print(f"6. ⚠️ Duplicate years found: {len(dup_years)} records")
        print(dup_years[['Year', 'Population']].sort_values('Year'))
    else:
        print(f"6. ✓ No duplicate years")
else:
    print("No population data available")

print("="*70)

POPULATION DATA QUALITY CHECKS:
1. Year range: 2013 to 2023
2. Negative populations: 0
3. Zero populations: 0
4. Population range: 316,128,839.0 to 334,914,896.0
5. ✓ No unusual year-over-year changes
6. ✓ No duplicate years


In [79]:
# 1. YEAR RANGE VALIDATION
print("1. YEAR RANGE VALIDATION:")
print(f"   Year range: {bls_data['year'].min()} to {bls_data['year'].max()}")
current_year = 2025
future_years = bls_data[bls_data['year'] > current_year]
if len(future_years) > 0:
    print(f"   ⚠️ Found {len(future_years)} records with future years")
old_years = bls_data[bls_data['year'] < 1947]  # BLS data typically starts 1947
if len(old_years) > 0:
    print(f"   ⚠️ Found {len(old_years)} records with years before 1947")
else:
    print("   ✓ All years within reasonable range")

# 2. PERIOD VALIDATION
print("\n2. PERIOD VALIDATION:")
valid_periods = ['Q01', 'Q02', 'Q03', 'Q04']
invalid_periods = bls_data[~bls_data['period'].isin(valid_periods)]
if len(invalid_periods) > 0:
    print(f"   ⚠️ Found {len(invalid_periods)} records with invalid periods:")
    print(f"   Invalid values: {invalid_periods['period'].unique()}")
else:
    print(f"   ✓ All periods are valid quarters: {valid_periods}")

# 3. VALUE OUTLIERS (using IQR method)
print("\n3. VALUE OUTLIERS:")
Q1 = bls_data['value'].quantile(0.25)
Q3 = bls_data['value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR
outliers = bls_data[(bls_data['value'] < lower_bound) | (bls_data['value'] > upper_bound)]
print(f"   IQR range: {lower_bound:.2f} to {upper_bound:.2f}")
print(f"   Outliers: {len(outliers):,} records ({len(outliers)/len(bls_data)*100:.2f}%)")
if len(outliers) > 0:
    print(f"   Min outlier: {outliers['value'].min():.2f}")
    print(f"   Max outlier: {outliers['value'].max():.2f}")

# 4. NEGATIVE/ZERO VALUES
print("\n4. NEGATIVE/ZERO VALUES:")
negative_count = (bls_data['value'] < 0).sum()
zero_count = (bls_data['value'] == 0).sum()
print(f"   Negative values: {negative_count:,} ({negative_count/len(bls_data)*100:.2f}%)")
print(f"   Zero values: {zero_count:,} ({zero_count/len(bls_data)*100:.2f}%)")
if negative_count > 0:
    print(f"   ⚠️ Check if negative values are expected for these series")

# 5. TEMPORAL COMPLETENESS (by series)
print("\n5. TEMPORAL COMPLETENESS:")
series_counts = bls_data.groupby('series_id').agg({
    'year': ['min', 'max', 'nunique'],
    'period': 'count'
}).reset_index()
series_counts.columns = ['series_id', 'min_year', 'max_year', 'unique_years', 'total_records']
series_counts['expected_records'] = (series_counts['max_year'] - series_counts['min_year'] + 1) * 4
series_counts['completeness_pct'] = (series_counts['total_records'] / series_counts['expected_records'] * 100)
incomplete = series_counts[series_counts['completeness_pct'] < 100]
print(f"   Series analyzed: {len(series_counts)}")
print(f"   Incomplete series: {len(incomplete)} ({len(incomplete)/len(series_counts)*100:.1f}%)")
if len(incomplete) > 0:
    print(f"   Average completeness: {incomplete['completeness_pct'].mean():.1f}%")
    print(f"   Most incomplete series: {incomplete.nsmallest(3, 'completeness_pct')[['series_id', 'completeness_pct']].to_dict('records')}")

# 6. QUARTER COMPLETENESS (each year should have 4 quarters)
print("\n6. QUARTER COMPLETENESS PER YEAR:")
year_quarter_counts = bls_data.groupby(['series_id', 'year'])['period'].nunique().reset_index()
year_quarter_counts.columns = ['series_id', 'year', 'quarter_count']
incomplete_years = year_quarter_counts[year_quarter_counts['quarter_count'] < 4]
print(f"   Total series-year combinations: {len(year_quarter_counts):,}")
print(f"   Incomplete years (< 4 quarters): {len(incomplete_years):,} ({len(incomplete_years)/len(year_quarter_counts)*100:.2f}%)")
if len(incomplete_years) > 0:
    print(f"   Sample incomplete years:")
    print(incomplete_years.head(5).to_string(index=False))

# 7. SERIES ID FORMAT VALIDATION
print("\n7. SERIES ID FORMAT:")
print(f"   Total unique series: {bls_data['series_id'].nunique()}")
# BLS series IDs are typically 17-20 characters
series_lengths = bls_data['series_id'].str.len()
print(f"   Series ID length range: {series_lengths.min()} to {series_lengths.max()} characters")
unusual_length = bls_data[(series_lengths < 10) | (series_lengths > 25)]
if len(unusual_length) > 0:
    print(f"   ⚠️ {unusual_length['series_id'].nunique()} series with unusual length")
else:
    print("   ✓ All series IDs have typical length")

print("\n" + "="*70)

1. YEAR RANGE VALIDATION:
   Year range: 1995 to 2025
   ✓ All years within reasonable range

2. PERIOD VALIDATION:
   ✓ All periods are valid quarters: ['Q01', 'Q02', 'Q03', 'Q04']

3. VALUE OUTLIERS:
   IQR range: -247.99 to 332.28
   Outliers: 1 records (0.00%)
   Min outlier: 384.60
   Max outlier: 384.60

4. NEGATIVE/ZERO VALUES:
   Negative values: 5,381 (18.61%)
   Zero values: 238 (0.82%)
   ⚠️ Check if negative values are expected for these series

5. TEMPORAL COMPLETENESS:
   Series analyzed: 237
   Incomplete series: 237 (100.0%)
   Average completeness: 98.4%
   Most incomplete series: [{'series_id': 'PRS30006011', 'completeness_pct': 98.38709677419355}, {'series_id': 'PRS30006012', 'completeness_pct': 98.38709677419355}, {'series_id': 'PRS30006013', 'completeness_pct': 98.38709677419355}]

6. QUARTER COMPLETENESS PER YEAR:
   Total series-year combinations: 7,347
   Incomplete years (< 4 quarters): 237 (3.23%)
   Sample incomplete years:
  series_id  year  quarter_count
PR

## Additional Data Quality Checks

## Load DataUSA Population Data

Load the latest population data from DataUSA API (JSON format with nested structure)

In [68]:
# List all population data files and get the latest
response = s3_client.list_objects_v2(
    Bucket=bucket_name,
    Prefix='raw/datausa/population/'
)
response

{'ResponseMetadata': {'RequestId': '8959YWZCK8PFJ7TM',
  'HostId': 'tUpz5EnGeEnDMnexenW/woPkI6s0GYk+YPWiQbzXuSyTRuiQ13rX0gWvs/5hWMmAZllkLx31XKodO1YBtXYYCQhDKyG1+/Ll',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'tUpz5EnGeEnDMnexenW/woPkI6s0GYk+YPWiQbzXuSyTRuiQ13rX0gWvs/5hWMmAZllkLx31XKodO1YBtXYYCQhDKyG1+/Ll',
   'x-amz-request-id': '8959YWZCK8PFJ7TM',
   'date': 'Mon, 22 Dec 2025 13:40:30 GMT',
   'x-amz-bucket-region': 'eu-north-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'raw/datausa/population/population_20251221_071535.json',
   'LastModified': datetime.datetime(2025, 12, 21, 7, 15, 36, tzinfo=tzutc()),
   'ETag': '"21da76ddc3780caf07f1ba2335f83212"',
   'ChecksumAlgorithm': ['CRC32'],
   'ChecksumType': 'FULL_OBJECT',
   'Size': 1844,
   'StorageClass': 'STANDARD'},
  {'Key': 'raw/datausa/population/population_20251221_075644.json',
   'LastMo

In [69]:
if 'Contents' in response:
    # Get the most recent file
    latest_file = sorted(response['Contents'], key=lambda x: x['LastModified'])[-1]
    pop_key = latest_file['Key']
    
    # Load JSON data
    obj = s3_client.get_object(Bucket=bucket_name, Key=pop_key)
    pop_json = json.loads(obj['Body'].read().decode('utf-8'))
    
    # Handle different JSON structures
    if 'data' in pop_json:
        # If JSON has a 'data' key, extract it
        pop_data = pd.DataFrame(pop_json['data'])
    elif isinstance(pop_json, list):
        # If JSON is a list, use it directly
        pop_data = pd.DataFrame(pop_json)
    else:
        # Otherwise try to use the JSON as-is
        pop_data = pd.DataFrame([pop_json])
    
    print(f"✓ Loaded population data from: {pop_key}")
    print(f"✓ Records: {len(pop_data):,}")
    print(f"✓ Columns: {list(pop_data.columns)}")
    print("\nPopulation data:")
    display(pop_data)
else:
    print("No population data found")
    pop_data = pd.DataFrame()

✓ Loaded population data from: raw/datausa/population/population_20251221_160017.json
✓ Records: 10
✓ Columns: ['Nation ID', 'Nation', 'Year', 'Population']

Population data:


,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0
5,01000US,United States,2018,327167439.0
6,01000US,United States,2019,328239523.0
7,01000US,United States,2021,331893745.0
8,01000US,United States,2022,333287562.0
9,01000US,United States,2023,334914896.0


## Q1

Calculate mean and standard deviation of population for years 2013-2018

In [70]:
# Filter for years 2013-2018
pop_filtered = pop_data[(pop_data['Year'] >= 2013) & (pop_data['Year'] <= 2018)].copy()
# Calculate statistics
pop_stats = pd.DataFrame({
    'Metric': ['Mean Population', 'Std Dev Population'],
    'Value': [pop_filtered['Population'].mean(), pop_filtered['Population'].std()]
})

print("POPULATION STATISTICS (2013-2018)")
print(pop_stats)
print(f"Population Analysis (2013-2018): Mean={pop_filtered['Population'].mean():.0f}, StdDev={pop_filtered['Population'].std():.0f}")
# Log results
import logging
logging.basicConfig(level=logging.INFO)
logging.info(f"Population Analysis (2013-2018): Mean={pop_filtered['Population'].mean():.0f}, StdDev={pop_filtered['Population'].std():.0f}")

INFO:root:Population Analysis (2013-2018): Mean=322069808, StdDev=4158441


POPULATION STATISTICS (2013-2018)
               Metric         Value
0     Mean Population  3.220698e+08
1  Std Dev Population  4.158441e+06
Population Analysis (2013-2018): Mean=322069808, StdDev=4158441


In [71]:
bls_data.groupby(['series_id', 'year'])['value'].sum()#.reset_index()

series_id    year
PRS30006011  1995      5.700
             1996     -0.400
             1997      3.500
             1998      3.400
             1999     -6.100
                      ...   
PRS88003203  2021    418.879
             2022    450.011
             2023    462.037
             2024    466.755
             2025    236.753
Name: value, Length: 7347, dtype: float64

##Q2

In [72]:
# Find best year per series (year with max sum of quarterly values)
yearly_sums = bls_data.groupby(['series_id', 'year'])['value'].sum().reset_index()
best_years_result = yearly_sums.loc[yearly_sums.groupby('series_id')['value'].idxmax()].sort_values('series_id').reset_index(drop=True)

print(f"Best years for {len(best_years_result):,} series:")
display(best_years_result)

# Log results - series_id as key, year and value as value
best_years_dict = best_years_result.set_index('series_id')[['year', 'value']].to_dict('index')
logging.info(f"Best Year Analysis: Found {len(best_years_result)} series, Top result: {best_years_dict}")
print(f"Best Year Analysis: Found {len(best_years_result)} series, Top result: {best_years_dict}")


Best years for 237 series:


,series_id,year,value
0,PRS30006011,2022,16.400
1,PRS30006012,2022,13.000
2,PRS30006013,1998,564.713
3,PRS30006021,2010,14.200
4,PRS30006022,2010,8.900
...,...,...,...
232,PRS88003192,2002,259.900
233,PRS88003193,2024,688.655
234,PRS88003201,2022,29.800
235,PRS88003202,2022,21.300


INFO:root:Best Year Analysis: Found 237 series, Top result: {'PRS30006011': {'year': 2022, 'value': 16.4}, 'PRS30006012': {'year': 2022, 'value': 13.0}, 'PRS30006013': {'year': 1998, 'value': 564.713}, 'PRS30006021': {'year': 2010, 'value': 14.2}, 'PRS30006022': {'year': 2010, 'value': 8.9}, 'PRS30006023': {'year': 2014, 'value': 402.512}, 'PRS30006031': {'year': 2022, 'value': 16.4}, 'PRS30006032': {'year': 2021, 'value': 13.9}, 'PRS30006033': {'year': 1998, 'value': 561.703}, 'PRS30006061': {'year': 2022, 'value': 29.599999999999998}, 'PRS30006062': {'year': 2021, 'value': 25.7}, 'PRS30006063': {'year': 2024, 'value': 517.4010000000001}, 'PRS30006091': {'year': 2002, 'value': 34.6}, 'PRS30006092': {'year': 2002, 'value': 35.699999999999996}, 'PRS30006093': {'year': 2013, 'value': 411.367}, 'PRS30006101': {'year': 2020, 'value': 26.7}, 'PRS30006102': {'year': 2020, 'value': 29.4}, 'PRS30006103': {'year': 2024, 'value': 516.052}, 'PRS30006111': {'year': 2020, 'value': 27.5}, 'PRS300061

Best Year Analysis: Found 237 series, Top result: {'PRS30006011': {'year': 2022, 'value': 16.4}, 'PRS30006012': {'year': 2022, 'value': 13.0}, 'PRS30006013': {'year': 1998, 'value': 564.713}, 'PRS30006021': {'year': 2010, 'value': 14.2}, 'PRS30006022': {'year': 2010, 'value': 8.9}, 'PRS30006023': {'year': 2014, 'value': 402.512}, 'PRS30006031': {'year': 2022, 'value': 16.4}, 'PRS30006032': {'year': 2021, 'value': 13.9}, 'PRS30006033': {'year': 1998, 'value': 561.703}, 'PRS30006061': {'year': 2022, 'value': 29.599999999999998}, 'PRS30006062': {'year': 2021, 'value': 25.7}, 'PRS30006063': {'year': 2024, 'value': 517.4010000000001}, 'PRS30006091': {'year': 2002, 'value': 34.6}, 'PRS30006092': {'year': 2002, 'value': 35.699999999999996}, 'PRS30006093': {'year': 2013, 'value': 411.367}, 'PRS30006101': {'year': 2020, 'value': 26.7}, 'PRS30006102': {'year': 2020, 'value': 29.4}, 'PRS30006103': {'year': 2024, 'value': 516.052}, 'PRS30006111': {'year': 2020, 'value': 27.5}, 'PRS30006112': {'yea

## Q3

In [73]:
# Filter for series PRS30006032, period Q01 and join with population
target_series, target_period = 'PRS30006032', 'Q01'
series_q01 = bls_data[(bls_data['series_id'].str.contains(target_series)) & (bls_data['period'] == target_period)].copy()

if len(pop_data) > 0:
    series_q01['year'] = series_q01['year'].astype(int)
    pop_data['Year'] = pop_data['Year'].astype(int)
    
    final_report = series_q01.merge(
        pop_data[['Year', 'Population']], 
        left_on='year', 
        right_on='Year', 
        how='left'
    )[['series_id', 'year', 'period', 'value', 'Population']].sort_values('year')
    
    print(f"Series {target_series} {target_period} + Population ({len(final_report)} rows):")
    display(final_report)
    
    # Log final report
    if len(final_report) > 0:
        year_range = f"{final_report['year'].min()}-{final_report['year'].max()}"
        final_report_dict = final_report.to_dict('records')
        logging.info(f"Analysis 3 - {target_series} {target_period}: Generated {len(final_report)} rows spanning {year_range}, Sample: {final_report_dict[:]}")
        print(f"Analysis 3 - {target_series} {target_period}: {len(final_report)} rows spanning {year_range}")
    else:
        logging.warning(f"Analysis 3 - {target_series} {target_period}: No matching data after merge")
else:
    final_report = series_q01
    logging.warning(f"Analysis 3 - {target_series} {target_period}: No population data available, returning {len(final_report)} BLS records only")
    print("⚠️ No population data available")

Series PRS30006032 Q01 + Population (31 rows):


,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.2,NaN
2,PRS30006032,1997,Q01,2.8,NaN
3,PRS30006032,1998,Q01,0.9,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.3,NaN
7,PRS30006032,2002,Q01,-6.6,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.0,NaN


INFO:root:Analysis 3 - PRS30006032 Q01: Generated 31 rows spanning 1995-2025, Sample: [{'series_id': 'PRS30006032', 'year': 1995, 'period': 'Q01', 'value': 0.0, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 1996, 'period': 'Q01', 'value': -4.2, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 1997, 'period': 'Q01', 'value': 2.8, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 1998, 'period': 'Q01', 'value': 0.9, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 1999, 'period': 'Q01', 'value': -4.1, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 2000, 'period': 'Q01', 'value': 0.5, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 2001, 'period': 'Q01', 'value': -6.3, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 2002, 'period': 'Q01', 'value': -6.6, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 2003, 'period': 'Q01', 'value': -5.7, 'Population': nan}, {'series_id': 'PRS30006032', 'year': 2004, 'period': 'Q01', 'v

Analysis 3 - PRS30006032 Q01: 31 rows spanning 1995-2025


In [74]:
# Diagnostic: Check all records for PRS30006032 Q01
print("Checking BLS data for series containing PRS30006032, period Q01:")
diagnostic = bls_data[(bls_data['series_id'].str.contains('PRS30006032')) & 
                      (bls_data['period'].str.contains('Q01'))]
print(f"Found {len(diagnostic)} matching rows:")
display(diagnostic)

# Log diagnostic results
if len(diagnostic) > 0:
    year_range = f"{diagnostic['year'].min()}-{diagnostic['year'].max()}"
    diagnostic_dict = diagnostic[['series_id', 'year', 'period', 'value']].to_dict('records')
    logging.info(f"Diagnostic - PRS30006032 Q01: Found {len(diagnostic)} rows spanning {year_range}, Data: {diagnostic_dict[:5]}")  # Log first 5 records
    print(f"Diagnostic - PRS30006032 Q01: Found {len(diagnostic)} rows spanning {year_range}")
else:
    logging.warning("Diagnostic - PRS30006032 Q01: No matching records found")
    print("⚠️ No matching records found")

Checking BLS data for series containing PRS30006032, period Q01:
Found 31 matching rows:


,series_id,year,period,value
1064,PRS30006032,1995,Q01,0.0
1069,PRS30006032,1996,Q01,-4.2
1074,PRS30006032,1997,Q01,2.8
1079,PRS30006032,1998,Q01,0.9
1084,PRS30006032,1999,Q01,-4.1
1089,PRS30006032,2000,Q01,0.5
1094,PRS30006032,2001,Q01,-6.3
1099,PRS30006032,2002,Q01,-6.6
1104,PRS30006032,2003,Q01,-5.7
1109,PRS30006032,2004,Q01,2.0


INFO:root:Diagnostic - PRS30006032 Q01: Found 31 rows spanning 1995-2025, Data: [{'series_id': 'PRS30006032', 'year': 1995, 'period': 'Q01', 'value': 0.0}, {'series_id': 'PRS30006032', 'year': 1996, 'period': 'Q01', 'value': -4.2}, {'series_id': 'PRS30006032', 'year': 1997, 'period': 'Q01', 'value': 2.8}, {'series_id': 'PRS30006032', 'year': 1998, 'period': 'Q01', 'value': 0.9}, {'series_id': 'PRS30006032', 'year': 1999, 'period': 'Q01', 'value': -4.1}]


Diagnostic - PRS30006032 Q01: Found 31 rows spanning 1995-2025
